# 下载公告

In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

def window_switch_to_new(driver, original_window):
    """跳转到新窗口"""
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            return
        
def window_switch_to_original(driver, original_window):
    """跳转到原始窗口"""
    driver.close()
    driver.switch_to.window(original_window)

def log_write(file, content):
    """写入日志文件"""
    with open(file, "a") as f:
        f.write(content + "\n")

# 日志文件, 记录已经下载的pdf文件
log = "data/log.txt"
saved = [f.strip() for f in open(log, "r").readlines()]

# 浏览器配置
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.prompt_for_download": False, # 禁用下载提示框
    "plugins.always_open_pdf_externally": True # 禁用 PDF Viewer 插件
}
chrome_options.add_experimental_option("prefs", prefs)

# 启动浏览器
driver = webdriver.Chrome(options=chrome_options)

进入检索目录

In [2]:
# 访问网页
driver.get("https://ggfw.cnipa.gov.cn/dlbzsq/dbQuery")

# 跳转到文件目录页面
# button = driver.find_element(By.XPATH, "/html/body/div/div/div[2]/div/div[2]/div/div[1]/button") # 新目录
button = driver.find_element(By.XPATH, "/html/body/div/div/div[2]/div/div[2]/div/div[2]/button") # 旧目录

button.click()
WebDriverWait(driver, 20)

# 记录原始窗口
original_window = driver.current_window_handle

爬取当前页面的所有文件

缺点：无法定位新页面，需要手动进入新页面

In [9]:
# 获取所有文件页面的链接
links = driver.find_elements(By.CLASS_NAME, "policyAnswersItem")

# 依次处理每一个文件
for link in links:
    # 跳转到新页面
    link.click()
    WebDriverWait(driver, 10)
    window_switch_to_new(driver, original_window)

    # PDF文件
    pdf_button = driver.find_element(By.CLASS_NAME, "el-link--inner")
    file_name = pdf_button.text
    
    # 判断是否要保存
    if file_name in saved:
        pass
    else:
        pdf_button.click()
        sleep(3)
        log_write(log, file_name)
        saved.append(file_name)

    # 跳转回起始页面
    window_switch_to_original(driver, original_window)
    sleep(1.5)

# 信息提取

使用大模型(Qwen3-Max)对公告信息进行提取，提示词如下。



你是一位专业的地理标志产品信息整理专家。请**严格依据我接下来提供的全部官方公告文本内容（包括正文及其所附的所有附件）**，为其中**每一个被批准实施地理标志（或原产地域）保护的产品**提取指定字段，并以 **标准 JSON 格式** 输出一个对象数组（`list of objects`）。每个对象必须且仅包含以下五个字段：

- `product_name`（产品名称）  
- `province`（省）  
- `city`（市）  
- `districts`（县/区，或具体乡镇列表）  
- `sensory_characteristics`（感官特征）

**提取与处理规则（务必逐条遵守）：**

1. **产品识别原则**：  
   - 每份公告**可能包含一个或多个**被保护的地理标志产品（例如标题或正文中明确列出多个产品名）。  
   - 你必须为**每一个独立产品**生成一个 JSON 对象。  
   - 判断依据：公告正文中“现批准自即日起对【XXX】实施……保护”中的【XXX】即为一个产品。若出现多次此类句式，则对应多个产品。
2. **product_name**：  
   - 必须**完全照抄**公告中用于申请保护的产品全称，不得缩写、合并或改写。
3. **province / city / districts**：  
   - **首先查阅公告正文中的“保护范围”部分；若该部分仅说明“见附件X”，则必须从对应附件中提取具体的产地范围描述。**  
   - 若附件或正文中明确写出县级及以上地名（如“南江县”“雷州市”“天台县”“大连市”），则：
     - `districts`：若列出具体乡镇，填入完整乡镇名数组（保留“乡”“镇”后缀）；若仅写“XX县现辖行政区域”，则填 `[ "XX县" ]`；若为“XX市现辖行政区域”且未列县，则填 `[]`。
     - `city` 和 `province`：**即使公告未明说**，也**允许你根据中国现行行政区划常识**推断并填写所属地级市和省级名称（例如：“雷州市” → city: "湛江市", province: "广东省"；“巴马瑶族自治县” → city: "河池市", province: "广西壮族自治区"）。
   - **严禁篡改或扩展公告原文的保护范围**。例如，公告写“伽师县的英买里乡……等13个乡镇”，你不能把 `districts` 写成 `["伽师县"]`，而应列出所有13个乡/镇名（若原文已枚举）；若原文只写“伽师县现辖行政区域”，则 `districts = ["伽师县"]`。
   - 若公告及附件中保护范围表述模糊到无法确定任何县级单位（极罕见），`districts` 可为空数组 `[]`，但仍需尽可能补全省、市。
4. **sensory_characteristics**：  
   - **必须优先查阅公告正文及所有附件**，寻找出现在“感官特色”“感官特征”“感观特征”“外观”“气味及滋味”“品质特色”等小标题下的**纯文字描述**。  
   - 若在**正文或附件中找到相关描述**，则**必须逐字引用原文**，禁止包含理化指标、加工方法等内容。  
   - **仅当公告正文及所有附件均未提供任何感官描述段落**（包括无“外观”“气味”“滋味”等相关文字），**才允许你联网搜索该地理标志产品的权威资料（如国家知识产权局官网、地方政府官网、农业农村部公告等）**，从中**客观、准确地提炼出公认的感官特征描述**，并以简洁、规范的中文写入该字段。  
   - **严禁在公告或附件已有感官描述的情况下自行补充或改写**；仅当**完全缺失**该部分内容时，才可启用联网搜索补充。  
   - 所有补充内容必须基于可靠来源，不得虚构、夸大或主观臆测。
5. **输出格式要求**：  
   - 整体输出必须是**合法、可直接解析的 JSON 数组**。  
   - 每个产品一个对象，顺序按其在公告中出现的先后排列。  
   - 所有字段必须存在，值可为空（如 `districts: []`）。  
   - **绝对禁止**包含任何额外内容：无解释、无注释、无序号、无“```json”、无总结语。


# 文本润色

由于表格等特殊信息，上述提取会产生文字错乱，因此使用大模型(Qwen3-Max)进一步润色，提示词如下。


你是一位专业的中文内容编辑与数据治理专家，擅长处理农产品、地理标志产品或地方特产的结构化信息。  

 **请对我提供的 JSON 数据执行以下综合操作**：
 对每个对象，依次完成 **字段补全 → 感官描述清洗与润色**，具体规则如下：

------

 **一、字段补全规则**（仅当字段为空、null、缺失或明显不完整时触发）  

  1. **`province` / `city` / `district`**：  
     - 若任一地理字段缺失，请**基于产品名称、常识或中国地理标志产品知识**进行合理推断并补全；  
     - 优先级：从产品名中提取地名（如“阳澄湖大闸蟹”→ 江苏/苏州/昆山）；若无法确定，使用最广为人知的主产区；  
     - 确保三级行政区划逻辑一致（如 province="广东"，则 city 不应为"杭州"）。
  2. **`sensory_characteristics`**：  
     - 若该字段为空、null 或仅含占位符（如“无”、“暂无”），请**根据产品类别和产地，生成一段典型、真实、符合实际的感官描述**；  
     - 描述需涵盖视觉、嗅觉、味觉、触觉中的至少两项；  
     - **不得编造不存在的特性**（如普通苹果不可写“带有玫瑰香气”），应基于该品类共性。

------

 **二、感官描述清洗与润色规则**（无论是否补全，均需执行）  

 - **删除所有量化指标**：如重量、尺寸、百分比、糖度、pH值、时间、温度等；  
 - **语言优化**：流畅、书面、简洁，突出自然感官体验；  
 - **风格统一**：客观专业，避免广告语、主观评价（如“天下第一”）；  
 - **不添加原文未暗示的属性**（除非是补全场景，且基于合理常识）。

------

 **三、格式与保全要求**  

 - **仅修改以下四个字段**：`province`、`city`、`district`、`sensory_characteristics`；  
 - 其他所有字段（如 `name`、`category` 等）**必须原样保留，不得改动或删除**；  
 - 输出为**合法、完整、可直接解析的 JSON 数组**，不要任何额外文本、解释、注释或 Markdown。

 —— 请处理以下数据 ——  

 ```json

 ```


# 信息整合

手动整合为一整个JSON文件:``data.json``

# 数据清洗

对``product_name``、``province``、``city``三个字段均相同的产品进行去重。

In [1]:
import os
os.chdir(os.getcwd())

import json

# 读取原始数据
with open('data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 使用集合记录已见过的 (product_name, province, city) 组合
seen = set()
unique_data = []

for item in data:
    # 安全提取字段，缺失时用 None 占位（保持一致性）
    name = item.get("product_name")
    province = item.get("province")
    city = item.get("city")
    
    # 构建唯一键：三元组
    key = (name, province, city)
    
    if key not in seen:
        seen.add(key)
        unique_data.append(item)

# 输出统计信息
print(f"原始条目数: {len(data)}")
print(f"去重后条目数: {len(unique_data)}")

# 保存清洗结果
with open('products.json', 'w', encoding='utf-8') as f:
    json.dump(unique_data, f, ensure_ascii=False, indent=2)

print("清洗完成！结果已保存为 products.json")

原始条目数: 4567
去重后条目数: 4404
清洗完成！结果已保存为 products.json


清洗非中国产品。

In [2]:
import os
os.chdir(os.getcwd())

import json

# 中国所有省级行政区（含港澳台）
CHINA_PROVINCES = {
    "北京市", "天津市", "上海市", "重庆市",
    "河北省", "山西省", "辽宁省", "吉林省", "黑龙江省",
    "江苏省", "浙江省", "安徽省", "福建省", "江西省",
    "山东省", "河南省", "湖北省", "湖南省", "广东省",
    "海南省", "四川省", "贵州省", "云南省", "陕西省",
    "甘肃省", "青海省", "台湾省",
    "内蒙古自治区", "广西壮族自治区", "西藏自治区",
    "宁夏回族自治区", "新疆维吾尔自治区",
    "香港特别行政区", "澳门特别行政区"
}

# 读取文件
with open('products.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 过滤：province 非空 且 属于中国
filtered_data = [
    item for item in data
    if item.get("province")  # 非空（排除 None, "", 等 falsy 值）
    and item["province"] in CHINA_PROVINCES
]

# 输出或保存
print(f"原始条目数: {len(data)}")
print(f"过滤后条目数: {len(filtered_data)}")

# 可选：写入新文件
with open('products.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

print("已保存到 products.json")

原始条目数: 4404
过滤后条目数: 4400
已保存到 products.json
